In [234]:
import pandas as pd
import numpy as np
import os
import glob

### Creamos el datamart final para entrenar el modelo

In [235]:
def load_homes_external(datalake_path):
    """
    Carga y une los archivos homes.tsv y external_indicators.tsv de todos los años en un único DataFrame.
    La unión se realiza por 'Año' y 'CCAA'.
    """
    all_homes = glob.glob(os.path.join(datalake_path, "*", "homes.tsv"))
    all_external = glob.glob(os.path.join(datalake_path, "*", "external_indicators.tsv"))

    df_list = []

    for home_file, external_file in zip(all_homes, all_external):
        year = os.path.basename(os.path.dirname(home_file))  # Extraer el año de la ruta

        df_homes = pd.read_csv(home_file, sep="\t")
        df_external = pd.read_csv(external_file, sep="\t")

        df_homes["Año"] = int(year)
        df_external["Año"] = int(year)

        # Unir por Año y CCAA
        df_merged = pd.merge(df_homes, df_external, on=["Año", "CCAA"], how="inner")
        df_list.append(df_merged)

    if not df_list:
        raise ValueError("No se encontraron archivos homes.tsv o external_indicators.tsv en la ruta del datalake.")

    # Concatenar todos los DataFrames
    df_homes_external = pd.concat(df_list, ignore_index=True)
    return df_homes_external

In [236]:
def merge_with_datamart(df_homes_external, datamart_path):
    """
    Une el DataFrame combinado de homes y external_indicators con el datamart de supercategorías.
    La unión se realiza por 'Año' y 'NUMERO'. Cada supercategoría se convierte en una columna
    con el gasto mensual correspondiente.
    """
    df_datamart = pd.read_csv(datamart_path, sep="\t")

    # Pivotar el datamart: cada SUPER_CATEGORIA como columna
    df_pivot = df_datamart.pivot_table(
        index=["Año", "NUMERO"],
        columns="SUPER_CATEGORIA",
        values="GASTO_MENSUAL",
    ).reset_index()

    # Renombrar columnas para mayor claridad, por ejemplo: SC_011, SC_012, etc.
    df_pivot.columns = ['Año', 'NUMERO'] + [f"SC_{col}" for col in df_pivot.columns[2:]]

    # Unir con el DataFrame combinado
    df_final = pd.merge(df_homes_external, df_pivot, on=["Año", "NUMERO"], how="inner")
    return df_final


In [237]:
load_homes_external_path = "DataLake" 
datamart_path = "datamarts/datamart_supercategories_three_digits.tsv"
output_path = "datamarts/final_dataframe.tsv" 

# Cargar y unir los archivos homes y external_indicators
df_homes_external = load_homes_external(load_homes_external_path)

# Unir con el datamart
df_final = merge_with_datamart(df_homes_external, datamart_path)


In [238]:
df_final.head(15)  # Mostrar las primeras filas del DataFrame final

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,EDADSP,OCUSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,CAPROP,CAJENA,IMPEXAC,GASTOT,FACTOR,NUMPERI,NACION_ESP,NACION_EXT,EDUC_SUPERIOR,NADUL_FEM,NADUL_MAS,NNINO_FEM,NNINO_MAS,Año,Comunidad_Autonoma,Tmax_max,Tmin_min,Tasa_Paro,Inflacion,Tipo_Interes,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_42,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128
0,1,7,0,1,1,2,2,0,66,1,5,3,96,1,1,0,1,0,1225,14239915.96,742.998897,2,1,0,0,1,1,0,0,2016,Castilla-La Mancha,42.9,-8.0,18.3,1.6,0.0,88.636140,0.076308,NaN,2.970634,18.523350,NaN,NaN,376.175600,NaN,21.250000,58.000000,NaN,NaN,NaN,NaN,NaN,4.436967,31.709100,6.759334,NaN,NaN,4.051743,4.813008,NaN,NaN,NaN,NaN,NaN,NaN,3.845126,1.417492,NaN,NaN,NaN,NaN,NaN,812.373168,NaN,120.268708,0.365467,NaN,39.583333,NaN,1.866225,NaN
1,47,7,0,1,1,1,0,0,85,0,6,1,100,1,1,0,0,0,910,3701813.56,410.584193,1,1,0,0,2,0,0,0,2016,Castilla-La Mancha,42.9,-8.0,18.3,1.6,0.0,180.604410,NaN,NaN,NaN,NaN,NaN,NaN,343.455701,NaN,6.020000,109.400930,NaN,NaN,41.612050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.079999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.157499,NaN,NaN,NaN
2,53,7,1,4,3,0,0,0,70,0,6,5,80,1,1,0,0,0,911,10433032.27,532.252963,1,1,0,0,1,1,0,0,2016,Castilla-La Mancha,42.9,-8.0,18.3,1.6,0.0,342.455008,11.949392,8.733919,NaN,69.451580,NaN,50.000000,458.742500,29.403101,141.566201,68.683800,NaN,NaN,NaN,7.097241,NaN,5.857157,NaN,NaN,NaN,NaN,84.514789,NaN,NaN,1.921005,28.600000,33.339775,NaN,11.514868,86.264533,NaN,NaN,NaN,NaN,NaN,NaN,135.956167,NaN,NaN,NaN,NaN,57.419168,NaN,NaN,NaN
3,67,7,1,4,2,1,1,0,25,1,4,5,85,1,1,0,0,1,450,1658015.51,385.268840,1,1,0,0,1,0,0,0,2016,Castilla-La Mancha,42.9,-8.0,18.3,1.6,0.0,20.839956,NaN,NaN,NaN,NaN,NaN,130.000000,NaN,NaN,6.720002,50.450001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.155027,64.240177,NaN,NaN,NaN,NaN,NaN,19.945490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.883172,NaN,NaN,4.393566,NaN,NaN,NaN,NaN,NaN
4,68,7,0,1,1,2,1,0,43,1,5,5,90,1,1,0,0,1,2800,39114856.55,1101.175074,2,1,0,0,1,1,0,1,2016,Castilla-La Mancha,42.9,-8.0,18.3,1.6,0.0,185.382973,4.506667,NaN,213.678822,170.688663,18.199356,NaN,309.110100,NaN,6.210000,147.050000,NaN,NaN,19.996700,NaN,NaN,33.620167,NaN,40.556000,NaN,NaN,153.983675,NaN,NaN,22.152000,68.000000,67.734000,NaN,260.742949,347.168596,NaN,NaN,97.500000,NaN,NaN,NaN,388.191885,194.330834,89.409213,NaN,NaN,40.760001,NaN,81.112000,NaN
5,69,7,1,5,3,1,1,0,68,0,6,5,85,1,1,0,1,0,1660,6744062.94,697.542695,2,1,0,1,1,1,0,0,2016,Castilla-La Mancha,42.9,-8.0,18.3,1.6,0.0,128.267157,16.639854,NaN,NaN,NaN,NaN,NaN,294.649400,NaN,102.759233,37.594100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.153467,NaN,NaN,NaN,48.370000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.180592,NaN,NaN,65.079166,NaN,NaN,NaN
6,76,7,1,4,2,1,0,1,59,0,6,6,74,1,1,0,0,0,1228,8268653.26,370.972834,1,1,0,1,1,0,0,1,2016,Castilla-La Mancha,42.9,-8.0,18.3,1.6,0.0,291.551034,36.716570,21.666232,NaN,133.833699,35.486208,NaN,424.336799,NaN,196.888898,63.511099,NaN,NaN,NaN,NaN,NaN,0.325893,NaN,108.149334,NaN,NaN,NaN,168.306017,NaN,50.048099,85.239999,NaN,NaN,NaN,NaN,9.763433,NaN,NaN,NaN,17.500001,NaN,15.598649,20.278001,84.669601,12.166700,NaN,40.833334,NaN,40.556000,NaN
7,94,7,1,4,3,1,1,0,57,1,6,5,89,1,1,1,0,1,1738,14991921.00,383.358779,1,1,0,0,1,1,0,0,2016,Castilla-La Mancha,42.9,-8.0,18.3,1.6,0.0,945.948825,35.594413,38.804022,NaN,74.014091,10.634001,NaN,780.333499,NaN,64.193330,68.619999,0.659035,NaN,NaN,NaN,NaN,91.811521,NaN,NaN,NaN,NaN,134.194799,NaN,NaN,NaN,54.220001,NaN,NaN,241.461139,NaN,4.837548,125.000000,NaN,NaN,NaN,NaN,210.039312,202.780000,109.696399,NaN,NaN,66.055000,NaN,NaN,NaN
8,109,7,0,1,1,2,2,0,36,1,5,6,198,1,1,0,0,1,3760,23950603.59,488.602611,2,1,0,1,1,1,0,0,2016,Castilla-La Mancha,42.9,-8.0,18.3,1.6,0.0,222.348631

Eliminamos la columna de Comunidad Autonoma ya que viene reflejada en la columna CCAA

In [239]:
df_final.drop(columns=["Comunidad_Autonoma", 'NACION_EXT'], inplace=True) 

In [240]:
df_final.head()  # Mostrar las primeras filas del DataFrame final

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,EDADSP,OCUSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,CAPROP,CAJENA,IMPEXAC,GASTOT,FACTOR,NUMPERI,NACION_ESP,EDUC_SUPERIOR,NADUL_FEM,NADUL_MAS,NNINO_FEM,NNINO_MAS,Año,Tmax_max,Tmin_min,Tasa_Paro,Inflacion,Tipo_Interes,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_42,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128
0,1,7,0,1,1,2,2,0,66,1,5,3,96,1,1,0,1,0,1225,14239915.96,742.998897,2,1,0,1,1,0,0,2016,42.9,-8.0,18.3,1.6,0.0,88.636140,0.076308,NaN,2.970634,18.523350,NaN,NaN,376.175600,NaN,21.250000,58.000000,NaN,NaN,NaN,NaN,NaN,4.436967,31.7091,6.759334,NaN,NaN,4.051743,4.813008,NaN,NaN,NaN,NaN,NaN,NaN,3.845126,1.417492,NaN,NaN,NaN,NaN,NaN,812.373168,NaN,120.268708,0.365467,NaN,39.583333,NaN,1.866225,NaN
1,47,7,0,1,1,1,0,0,85,0,6,1,100,1,1,0,0,0,910,3701813.56,410.584193,1,1,0,2,0,0,0,2016,42.9,-8.0,18.3,1.6,0.0,180.604410,NaN,NaN,NaN,NaN,NaN,NaN,343.455701,NaN,6.020000,109.400930,NaN,NaN,41.61205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.079999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.157499,NaN,NaN,NaN
2,53,7,1,4,3,0,0,0,70,0,6,5,80,1,1,0,0,0,911,10433032.27,532.252963,1,1,0,1,1,0,0,2016,42.9,-8.0,18.3,1.6,0.0,342.455008,11.949392,8.733919,NaN,69.451580,NaN,50.0,458.742500,29.403101,141.566201,68.683800,NaN,NaN,NaN,7.097241,NaN,5.857157,NaN,NaN,NaN,NaN,84.514789,NaN,NaN,1.921005,28.600000,33.339775,NaN,11.514868,86.264533,NaN,NaN,NaN,NaN,NaN,NaN,135.956167,NaN,NaN,NaN,NaN,57.419168,NaN,NaN,NaN
3,67,7,1,4,2,1,1,0,25,1,4,5,85,1,1,0,0,1,450,1658015.51,385.268840,1,1,0,1,0,0,0,2016,42.9,-8.0,18.3,1.6,0.0,20.839956,NaN,NaN,NaN,NaN,NaN,130.0,NaN,NaN,6.720002,50.450001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.155027,64.240177,NaN,NaN,NaN,NaN,NaN,19.945490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.883172,NaN,NaN,4.393566,NaN,NaN,NaN,NaN,NaN
4,68,7,0,1,1,2,1,0,43,1,5,5,90,1,1,0,0,1,2800,39114856.55,1101.175074,2,1,0,1,1,0,1,2016,42.9,-8.0,18.3,1.6,0.0,185.382973,4.506667,NaN,213.678822,170.688663,18.199356,NaN,309.110100,NaN,6.210000,147.050000,NaN,NaN,19.99670,NaN,NaN,33.620167,NaN,40.556000,NaN,NaN,153.983675,NaN,NaN,22.152000,68.000000,67.734000,NaN,260.742949,347.168596,NaN,NaN,97.5,NaN,NaN,NaN,388.191885,194.330834,89.409213,NaN,NaN,40.760001,NaN,81.112000,NaN


### Calculamos la inflación acumulada y la aplicamos a todos las columnas que sean tipo €

In [241]:
inflacion_df = pd.read_csv('datos\inflacion_espana_2016_2023.csv', sep='\t')

In [242]:
codigos_comunidad = {
    "Andalucía": 1, "Aragón": 2, "Asturias, Principado de": 3, "Balears, Illes": 4,
    "Canarias": 5, "Cantabria": 6, "Castilla y León": 8, "Castilla-La Mancha": 7,
    "Cataluña": 9, "Comunitat Valenciana": 10, "Extremadura": 11, "Galicia": 12,
    "Madrid, Comunidad de": 13, "Murcia, Región de": 14, "Navarra, Comunidad Foral de": 15,
    "País Vasco": 16, "Rioja, La": 17, "Ceuta": 18, "Melilla": 19
}

# Agregar código de comunidad
inflacion_df["CCAA"] = inflacion_df["Comunidad Autónoma"].map(codigos_comunidad)


In [243]:
inflacion_df.head()

,Año,Comunidad Autónoma,Inflación (%),CCAA
0,2016,Nacional,1.6,NaN
1,2016,Andalucía,1.6,1.0
2,2016,Aragón,1.6,2.0
3,2016,"Asturias, Principado de",1.6,3.0
4,2016,"Balears, Illes",1.6,4.0


In [244]:
def calcular_factores_acumulados_a_2023(df_inflacion):
    """
    Dado un DataFrame con columnas: 'Año', 'Comunidad Autónoma', 'Inflación (%)', 'CCAA',
    devuelve un DataFrame con la inflación acumulada para ajustar precios a 2023.
    """
    df = df_inflacion.copy()
    df = df.sort_values(['CCAA', 'Año'])

    # A porcentaje decimal
    df['inflacion_decimal'] = df['Inflación (%)'] / 100

    # Calcular factor acumulado por CCAA
    df['factor_acumulado'] = df.groupby('CCAA')['inflacion_decimal'].transform(lambda x: (1 + x).cumprod())

    # Obtener el factor en 2023 para cada CCAA
    factores_2023 = df[df['Año'] == 2023].set_index('CCAA')['factor_acumulado'].to_dict()

    # Crear factor de ajuste a 2023: factor_acumulado / factor_2023
    df['factor_ajuste_2023'] = df.apply(
        lambda row: row['factor_acumulado'] / factores_2023.get(row['CCAA'], np.nan),
        axis=1
    )

    return df[['Año', 'Comunidad Autónoma', 'CCAA', 'factor_ajuste_2023']]


In [245]:
inflacion_ajuste = calcular_factores_acumulados_a_2023(inflacion_df)
# Eliminar filas donde Comunidad Autónoma es Nacional
inflacion_ajuste = inflacion_ajuste[inflacion_ajuste['Comunidad Autónoma'] != 'Nacional']

inflacion_ajuste

,Año,Comunidad Autónoma,CCAA,factor_ajuste_2023
1,2016,Andalucía,1.0,0.834982
21,2017,Andalucía,1.0,0.844167
41,2018,Andalucía,1.0,0.852608
61,2019,Andalucía,1.0,0.856871
81,2020,Andalucía,1.0,0.854301
...,...,...,...,...
79,2019,Melilla,19.0,0.847833
99,2020,Melilla,19.0,0.848681
119,2021,Melilla,19.0,0.907240
139,2022,Melilla,19.0,0.967118


In [246]:
def aplicar_ajuste_inflacion(df_final, df_factores):
    """
    Aplica los factores de ajuste a 2023 a las columnas monetarias del df_final,
    modificando directamente las columnas que comienzan por 'SC_' más 'GASTOT' e 'IMPEXAC'.
    """
    df = df_final.copy()

    # Unir el factor a cada fila según Año y CCAA
    df = df.merge(df_factores[['Año', 'CCAA', 'factor_ajuste_2023']],
                  on=['Año', 'CCAA'], how='left')

    # Columnas monetarias a ajustar
    columnas_monetarias = [col for col in df.columns if col.startswith('SC_')]
    columnas_monetarias += [col for col in ['GASTOT', 'IMPEXAC'] if col in df.columns]

    # Ajustar directamente las columnas (manteniendo NaNs)
    for col in columnas_monetarias:
        df[col] = df[col] * df['factor_ajuste_2023']

    # Eliminar columna auxiliar
    df.drop(columns=['factor_ajuste_2023'], inplace=True)

    return df


In [247]:
df_final_ajustado = aplicar_ajuste_inflacion(df_final, inflacion_ajuste)
# Mostrar la casa NUMERO 1 en 2016, para df_final
df_final[df_final['NUMERO'] == 1][df_final['Año'] == 2016].head(15)

C:\Users\rauul\AppData\Local\Temp\ipykernel_8204\258148125.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_final[df_final['NUMERO'] == 1][df_final['Año'] == 2016].head(15)


,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,EDADSP,OCUSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,CAPROP,CAJENA,IMPEXAC,GASTOT,FACTOR,NUMPERI,NACION_ESP,EDUC_SUPERIOR,NADUL_FEM,NADUL_MAS,NNINO_FEM,NNINO_MAS,Año,Tmax_max,Tmin_min,Tasa_Paro,Inflacion,Tipo_Interes,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_42,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128
0,1,7,0,1,1,2,2,0,66,1,5,3,96,1,1,0,1,0,1225,14239915.96,742.998897,2,1,0,1,1,0,0,2016,42.9,-8.0,18.3,1.6,0.0,88.63614,0.076308,NaN,2.970634,18.52335,NaN,NaN,376.1756,NaN,21.25,58.0,NaN,NaN,NaN,NaN,NaN,4.436967,31.7091,6.759334,NaN,NaN,4.051743,4.813008,NaN,NaN,NaN,NaN,NaN,NaN,3.845126,1.417492,NaN,NaN,NaN,NaN,NaN,812.373168,NaN,120.268708,0.365467,NaN,39.583333,NaN,1.866225,NaN


Pasamos el año a timestamp

In [248]:
df_final_ajustado[df_final_ajustado['NUMERO'] == 1][df_final_ajustado['Año'] == 2016].head(15)  # Mostrar la casa NUMERO 1 en 2016, para df_final_ajustado

C:\Users\rauul\AppData\Local\Temp\ipykernel_8204\1241866064.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_final_ajustado[df_final_ajustado['NUMERO'] == 1][df_final_ajustado['Año'] == 2016].head(15)  # Mostrar la casa NUMERO 1 en 2016, para df_final_ajustado


,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,EDADSP,OCUSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,CAPROP,CAJENA,IMPEXAC,GASTOT,FACTOR,NUMPERI,NACION_ESP,EDUC_SUPERIOR,NADUL_FEM,NADUL_MAS,NNINO_FEM,NNINO_MAS,Año,Tmax_max,Tmin_min,Tasa_Paro,Inflacion,Tipo_Interes,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_42,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128
0,1,7,0,1,1,2,2,0,66,1,5,3,96,1,1,0,1,0,1007.709928,1.171404e+07,742.998897,2,1,0,1,1,0,0,2016,42.9,-8.0,18.3,1.6,0.0,72.913892,0.062772,NaN,2.443704,15.237684,NaN,NaN,309.449703,NaN,17.480683,47.71198,NaN,NaN,NaN,NaN,NaN,3.649939,26.084551,5.560365,NaN,NaN,3.333047,3.959279,NaN,NaN,NaN,NaN,NaN,NaN,3.163079,1.166058,NaN,NaN,NaN,NaN,NaN,668.274699,NaN,98.935487,0.300641,NaN,32.562055,NaN,1.535195,NaN


In [249]:
df_final_ajustado['Timestamp'] = pd.to_datetime(df_final_ajustado['Año'].astype(str) + '-01-01T00:00:00')
df_final_ajustado['Timestamp'] = df_final_ajustado['Timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# Borrar la columna 'Año' si no es necesaria
df_final_ajustado.drop(columns=['Año'], inplace=True)

df_final_ajustado['Timestamp'].head()  

0    2016-01-01T00:00:00
1    2016-01-01T00:00:00
2    2016-01-01T00:00:00
3    2016-01-01T00:00:00
4    2016-01-01T00:00:00
Name: Timestamp, dtype: object

### Dividir el GASTOT entre el factor poblacional

In [250]:
### Dividir el GASTOT entre el factor poblacional Y PASARLO DE AÑO A MES
df_final_ajustado['GASTOT_DIV_P'] = df_final_ajustado['GASTOT'] / df_final_ajustado['FACTOR']
df_final_ajustado['GASTOT_MENSUAL'] = df_final_ajustado['GASTOT_DIV_P'] / 12  # Pasar de anual a mensual

# Borrar la columna 'GASTOT_DIV_P' si no es necesaria
df_final_ajustado.drop(columns=['GASTOT_DIV_P', 'GASTOT'], inplace=True)

In [251]:
df_final_ajustado.head()  # Mostrar las primeras filas de la columna GASTOT ajustada

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,EDADSP,OCUSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,CAPROP,CAJENA,IMPEXAC,FACTOR,NUMPERI,NACION_ESP,EDUC_SUPERIOR,NADUL_FEM,NADUL_MAS,NNINO_FEM,NNINO_MAS,Tmax_max,Tmin_min,Tasa_Paro,Inflacion,Tipo_Interes,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_42,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128,Timestamp,GASTOT_MENSUAL
0,1,7,0,1,1,2,2,0,66,1,5,3,96,1,1,0,1,0,1007.709928,742.998897,2,1,0,1,1,0,0,42.9,-8.0,18.3,1.6,0.0,72.913892,0.062772,NaN,2.443704,15.237684,NaN,NaN,309.449703,NaN,17.480683,47.711980,NaN,NaN,NaN,NaN,NaN,3.649939,26.084551,5.560365,NaN,NaN,3.333047,3.959279,NaN,NaN,NaN,NaN,NaN,NaN,3.163079,1.166058,NaN,NaN,NaN,NaN,NaN,668.274699,NaN,98.935487,0.300641,NaN,32.562055,NaN,1.535195,NaN,2016-01-01T00:00:00,1313.824812
1,47,7,0,1,1,1,0,0,85,0,6,1,100,1,1,0,0,0,748.584518,410.584193,1,1,0,2,0,0,0,42.9,-8.0,18.3,1.6,0.0,148.568863,NaN,NaN,NaN,NaN,NaN,NaN,282.533648,NaN,4.952174,89.995431,NaN,NaN,34.230919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.502762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.276033,NaN,NaN,NaN,2016-01-01T00:00:00,618.059831
2,53,7,1,4,3,0,0,0,70,0,6,5,80,1,1,0,0,0,749.407138,532.252963,1,1,0,1,1,0,0,42.9,-8.0,18.3,1.6,0.0,281.710458,9.829813,7.184699,NaN,57.132283,NaN,41.131018,377.370916,24.187589,116.455238,56.500691,NaN,NaN,NaN,5.838335,NaN,4.818217,NaN,NaN,NaN,NaN,69.523585,NaN,NaN,1.580258,23.526942,27.425978,NaN,9.472365,70.962960,NaN,NaN,NaN,NaN,NaN,NaN,111.840309,NaN,NaN,NaN,NaN,47.234176,NaN,NaN,NaN,2016-01-01T00:00:00,1343.725829
3,67,7,1,4,2,1,1,0,25,1,4,5,85,1,1,0,0,1,370.179157,385.268840,1,1,0,1,0,0,0,42.9,-8.0,18.3,1.6,0.0,17.143372,NaN,NaN,NaN,NaN,NaN,106.940645,NaN,NaN,5.528011,41.501197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.016888,52.845277,NaN,NaN,NaN,NaN,NaN,16.407566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.016997,NaN,NaN,3.614237,NaN,NaN,NaN,NaN,NaN,2016-01-01T00:00:00,295.014190
4,68,7,0,1,1,2,1,0,43,1,5,5,90,1,1,0,0,1,2303.336977,1101.175074,2,1,0,1,1,0,1,42.9,-8.0,18.3,1.6,0.0,152.499806,3.707276,NaN,175.776547,140.411967,14.97116,NaN,254.280259,NaN,5.108473,120.966323,NaN,NaN,16.449692,NaN,NaN,27.656634,NaN,33.362191,NaN,NaN,126.670105,NaN,NaN,18.222686,55.938184,55.719367,NaN,214.492456,285.587951,NaN,NaN,80.205484,NaN,NaN,NaN,319.334544,159.860498,73.549838,NaN,NaN,33.530006,NaN,66.724382,NaN,2016-01-01T00:00:00,2435.025827


### Renombramos las columnas de gasto mensual por categorías para que se entienda

In [252]:
def renombrar_columnas_sc(df, categoria_detallada):
    """
    Renombra las columnas SC_{codigo} a una abreviatura legible basada en el nombre de la categoría.
    Ejemplo: SC_011 → ALIMENTOS_011
    """
    df = df.copy()
    columnas_renombradas = {}

    for col in df.columns:
        if col.startswith("SC_"):
            codigo = col.split("_")[1]
            nombre_categoria = categoria_detallada.get(codigo.lstrip("0"), None)

            if nombre_categoria:
                # Tomamos una abreviatura clara: primeras palabras en mayúsculas, separadas por _
                abreviatura = (
                    nombre_categoria.upper()
                    .replace(",", "")
                    .replace(".", "")
                    .replace("/", "_")
                    .replace(" Y ", "_")
                    .replace(" ", "_")
                )
                nuevo_nombre = f"{abreviatura}_{codigo}"
                columnas_renombradas[col] = nuevo_nombre

    return df.rename(columns=columnas_renombradas)

In [253]:
# Diccionario de nombres sin puntos
categoria_detallada = {
        "11": "PRODUCTOS ALIMENTICIOS",
        "12": "BEBIDAS NO ALCOHÓLICAS",
        "21": "BEBIDAS ALCOHÓLICAS",
        "22": "TABACO",
        "31": "ARTÍCULOS DE VESTIR",
        "32": "CALZADO",
        "41": "ALQUILERES REALES DE LA VIVIENDA",
        "42": "ALQUILERES IMPUTADOS DE LA VIVIENDA",
        "43": "MANTENIMIENTO DE LA VIVIENDA",
        "44": "SUMINISTRO DE AGUA",
        "45": "ELECTRICIDAD, GAS Y OTROS COMBUSTIBLES",
        "51": "MUEBLES Y REVESTIMIENTOS",
        "52": "TEXTILES PARA EL HOGAR",
        "53": "GRANDES ELECTRODOMÉSTICOS",
        "54": "UTENSILIOS DEL HOGAR",
        "55": "HERRAMIENTAS CASA/JARDÍN",
        "56": "BIENES Y SERVICIOS PARA EL HOGAR",
        "61": "PRODUCTOS FARMACÉUTICOS",
        "62": "SERVICIOS MÉDICOS AMBULATORIOS",
        "63": "SERVICIOS HOSPITALARIOS",
        "71": "COMPRA DE VEHÍCULOS",
        "72": "USO DE VEHÍCULOS PERSONALES",
        "73": "SERVICIOS DE TRANSPORTE",
        "81": "SERVICIOS POSTALES",
        "82": "EQUIPOS TELÉFONO/FAX",
        "83": "SERVICIOS TELÉFONO/FAX",
        "91": "AUDIOVISUALES Y TECNOLOGÍA",
        "92": "BIENES DURADEROS DE OCIO",
        "93": "OCIO, JARDINERÍA Y MASCOTAS",
        "94": "SERVICIOS RECREATIVOS Y CULTURALES",
        "95": "PRENSA Y PAPELERÍA",
        "96": "PAQUETES TURÍSTICOS",
        "101": "EDUCACIÓN INFANTIL Y PRIMARIA",
        "102": "EDUCACIÓN SECUNDARIA Y POSTSECUNDARIA",
        "103": "EDUCACIÓN SUPERIOR",
        "104": "EDUCACIÓN NO FORMAL",
        "111": "RESTAURACIÓN",
        "112": "ALOJAMIENTO",
        "121": "CUIDADOS PERSONALES",
        "123": "EFECTOS PERSONALES",
        "124": "PROTECCIÓN SOCIAL",
        "125": "SEGUROS",
        "126": "SERVICIOS FINANCIEROS",
        "127": "OTROS SERVICIOS",
        "128": "REMESAS"
    }

df_final_ajustado = renombrar_columnas_sc(df_final_ajustado, categoria_detallada)

In [254]:
df_final_ajustado

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,EDADSP,OCUSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,DISPOSIOV,CAPROP,CAJENA,IMPEXAC,FACTOR,NUMPERI,NACION_ESP,EDUC_SUPERIOR,NADUL_FEM,NADUL_MAS,NNINO_FEM,NNINO_MAS,Tmax_max,Tmin_min,Tasa_Paro,Inflacion,Tipo_Interes,PRODUCTOS_ALIMENTICIOS_11,BEBIDAS_NO_ALCOHÓLICAS_12,BEBIDAS_ALCOHÓLICAS_21,TABACO_22,ARTÍCULOS_DE_VESTIR_31,CALZADO_32,ALQUILERES_REALES_DE_LA_VIVIENDA_41,ALQUILERES_IMPUTADOS_DE_LA_VIVIENDA_42,MANTENIMIENTO_DE_LA_VIVIENDA_43,SUMINISTRO_DE_AGUA_44,ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_45,MUEBLES_REVESTIMIENTOS_51,TEXTILES_PARA_EL_HOGAR_52,GRANDES_ELECTRODOMÉSTICOS_53,UTENSILIOS_DEL_HOGAR_54,HERRAMIENTAS_CASA_JARDÍN_55,BIENES_SERVICIOS_PARA_EL_HOGAR_56,PRODUCTOS_FARMACÉUTICOS_61,SERVICIOS_MÉDICOS_AMBULATORIOS_62,SERVICIOS_HOSPITALARIOS_63,COMPRA_DE_VEHÍCULOS_71,USO_DE_VEHÍCULOS_PERSONALES_72,SERVICIOS_DE_TRANSPORTE_73,SERVICIOS_POSTALES_81,EQUIPOS_TELÉFONO_FAX_82,SERVICIOS_TELÉFONO_FAX_83,AUDIOVISUALES_TECNOLOGÍA_91,BIENES_DURADEROS_DE_OCIO_92,OCIO_JARDINERÍA_MASCOTAS_93,SERVICIOS_RECREATIVOS_CULTURALES_94,PRENSA_PAPELERÍA_95,PAQUETES_TURÍSTICOS_96,EDUCACIÓN_INFANTIL_PRIMARIA_101,EDUCACIÓN_SECUNDARIA_POSTSECUNDARIA_102,EDUCACIÓN_SUPERIOR_103,EDUCACIÓN_NO_FORMAL_104,RESTAURACIÓN_111,ALOJAMIENTO_112,CUIDADOS_PERSONALES_121,EFECTOS_PERSONALES_123,PROTECCIÓN_SOCIAL_124,SEGUROS_125,SERVICIOS_FINANCIEROS_126,OTROS_SERVICIOS_127,REMESAS_128,Timestamp,GASTOT_MENSUAL
0,1,7,0,1,1,2,2,0,66,1,5,3,96,1,1,0,1,0,1007.709928,742.998897,2,1,0,1,1,0,0,42.9,-8.0,18.3,1.6,0.0,72.913892,0.062772,NaN,2.443704,15.237684,NaN,NaN,309.449703,NaN,17.480683,47.711980,NaN,NaN,NaN,NaN,NaN,3.649939,26.084551,5.560365,NaN,NaN,3.333047,3.959279,NaN,NaN,NaN,NaN,NaN,NaN,3.163079,1.166058,NaN,NaN,NaN,NaN,NaN,668.274699,NaN,98.935487,0.300641,NaN,32.562055,NaN,1.535195,NaN,2016-01-01T00:00:00,1313.824812
1,47,7,0,1,1,1,0,0,85,0,6,1,100,1,1,0,0,0,748.584518,410.584193,1,1,0,2,0,0,0,42.9,-8.0,18.3,1.6,0.0,148.568863,NaN,NaN,NaN,NaN,NaN,NaN,282.533648,NaN,4.952174,89.995431,NaN,NaN,34.230919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.502762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.276033,NaN,NaN,NaN,2016-01-01T00:00:00,618.059831
2,53,7,1,4,3,0,0,0,70,0,6,5,80,1,1,0,0,0,749.407138,532.252963,1,1,0,1,1,0,0,42.9,-8.0,18.3,1.6,0.0,281.710458,9.829813,7.184699,NaN,57.132283,NaN,41.131018,377.370916,24.187589,116.455238,56.500691,NaN,NaN,NaN,5.838335,NaN,4.818217,NaN,NaN,NaN,NaN,69.523585,NaN,NaN,1.580258,23.526942,27.425978,NaN,9.472365,70.962960,NaN,NaN,NaN,NaN,NaN,NaN,111.840309,NaN,NaN,NaN,NaN,47.234176,NaN,NaN,NaN,2016-01-01T00:00:00,1343.725829
3,67,7,1,4,2,1,1,0,25,1,4,5,85,1,1,0,0,1,370.179157,385.268840,1,1,0,1,0,0,0,42.9,-8.0,18.3,1.6,0.0,17.143372,NaN,NaN,NaN,NaN,NaN,106.940645,NaN,NaN,5.528011,41.501197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.016888,52.845277,NaN,NaN,NaN,NaN,NaN,16.407566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.016997,NaN,NaN,3.614237,NaN,NaN,NaN,NaN,NaN,2016-01-01T00:00:00,295.014190
4,68,7,0,1,1,2,1,0,43,1,5,5,90,1,1,0,0,1,2303.336977,1101.175074,2,1,0,1,1,0,1,42.9,-8.0,18.3,1.6,0.0,152.499806,3.707276,NaN,175.776547,140.411967,14.971160,NaN,254.280259,NaN,5.108473,120.966323,NaN,NaN,16.449692,NaN,NaN,27.656634,NaN,33.362191,NaN,NaN,126.670105,NaN,NaN,18.222686,55.938184,55.719367,NaN,214.492456,285.587951,NaN,NaN,80.205484,NaN,NaN,NaN,319.334544,159.860498,73.549838,NaN,NaN,33.530006,NaN,66.724382,NaN,2016-01-01T00:00:00,2435.025827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166015,19940,18,1,4,3,4,0,0,43,0,3,4,82,1,0,0,0,0,1080.000000,337.038282,2,0,0,2,2,0,4,40.8,7.1,24.0,3.3,4.5,470.500593,125.299387,NaN,NaN,730.002000,121.667000,98.000000,NaN,NaN,26.000001,44.999999,NaN,NaN,NaN,NaN

### Ordenamos las columnas de nuestro dataframe

In [255]:
columnas_prioritarias = [
    "Timestamp", "NUMERO", "CCAA", "FACTOR", "CAPROV", "TAMAMU", "DENSIDAD",
    "SUPERF", "AGUACALI", "CALEF", "ZONARES", "REGTEN", "NUMOCU", "NUMACTI",
    "NUMPERI", "NUMESTU", "NADUL_MAS", "NADUL_FEM", "NNINO_FEM", "NNINO_MAS",
    "OCUSP", "EDADSP", "NACION_ESP", "EDUC_SUPERIOR", "CAPROP", "CAJENA",
    "DISPOSIOV", "IMPEXAC", "GASTOT_MENSUAL"
]


In [256]:
def reordenar_columnas(df, columnas_prioritarias):
    # Asegurar que solo usamos las que realmente están en el df
    columnas_prioritarias = [col for col in columnas_prioritarias if col in df.columns]
    
    # Identificar columnas que no están en la lista prioritaria
    columnas_restantes = [col for col in df.columns if col not in columnas_prioritarias]
    
    # Orden final
    orden_final = columnas_prioritarias + columnas_restantes
    
    return df[orden_final]


In [257]:
df = reordenar_columnas(df_final_ajustado, columnas_prioritarias)


In [264]:
df['CCAA'].value_counts()  # Mostrar los números únicos de las casas

1     18084
16    17312
9     14472
10    13218
13    12924
7     10805
12    10558
8      8838
5      7695
11     7507
2      7398
14     7305
3      7118
6      5986
15     5454
17     5240
4      4440
18      882
19      784
Name: CCAA, dtype: int64

In [265]:
df.head(5)  # Mostrar las primeras filas del DataFrame reordenado

,Timestamp,NUMERO,CCAA,FACTOR,CAPROV,TAMAMU,DENSIDAD,SUPERF,AGUACALI,CALEF,ZONARES,REGTEN,NUMOCU,NUMACTI,NUMPERI,NUMESTU,NADUL_MAS,NADUL_FEM,NNINO_FEM,NNINO_MAS,OCUSP,EDADSP,NACION_ESP,EDUC_SUPERIOR,CAPROP,CAJENA,DISPOSIOV,IMPEXAC,GASTOT_MENSUAL,Tmax_max,Tmin_min,Tasa_Paro,Inflacion,Tipo_Interes,PRODUCTOS_ALIMENTICIOS_11,BEBIDAS_NO_ALCOHÓLICAS_12,BEBIDAS_ALCOHÓLICAS_21,TABACO_22,ARTÍCULOS_DE_VESTIR_31,CALZADO_32,ALQUILERES_REALES_DE_LA_VIVIENDA_41,ALQUILERES_IMPUTADOS_DE_LA_VIVIENDA_42,MANTENIMIENTO_DE_LA_VIVIENDA_43,SUMINISTRO_DE_AGUA_44,ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_45,MUEBLES_REVESTIMIENTOS_51,TEXTILES_PARA_EL_HOGAR_52,GRANDES_ELECTRODOMÉSTICOS_53,UTENSILIOS_DEL_HOGAR_54,HERRAMIENTAS_CASA_JARDÍN_55,BIENES_SERVICIOS_PARA_EL_HOGAR_56,PRODUCTOS_FARMACÉUTICOS_61,SERVICIOS_MÉDICOS_AMBULATORIOS_62,SERVICIOS_HOSPITALARIOS_63,COMPRA_DE_VEHÍCULOS_71,USO_DE_VEHÍCULOS_PERSONALES_72,SERVICIOS_DE_TRANSPORTE_73,SERVICIOS_POSTALES_81,EQUIPOS_TELÉFONO_FAX_82,SERVICIOS_TELÉFONO_FAX_83,AUDIOVISUALES_TECNOLOGÍA_91,BIENES_DURADEROS_DE_OCIO_92,OCIO_JARDINERÍA_MASCOTAS_93,SERVICIOS_RECREATIVOS_CULTURALES_94,PRENSA_PAPELERÍA_95,PAQUETES_TURÍSTICOS_96,EDUCACIÓN_INFANTIL_PRIMARIA_101,EDUCACIÓN_SECUNDARIA_POSTSECUNDARIA_102,EDUCACIÓN_SUPERIOR_103,EDUCACIÓN_NO_FORMAL_104,RESTAURACIÓN_111,ALOJAMIENTO_112,CUIDADOS_PERSONALES_121,EFECTOS_PERSONALES_123,PROTECCIÓN_SOCIAL_124,SEGUROS_125,SERVICIOS_FINANCIEROS_126,OTROS_SERVICIOS_127,REMESAS_128
0,2016-01-01T00:00:00,1,7,742.998897,0,1,1,96,1,1,3,5,2,2,2,0,1,1,0,0,1,66,1,0,1,0,0,1007.709928,1313.824812,42.9,-8.0,18.3,1.6,0.0,72.913892,0.062772,NaN,2.443704,15.237684,NaN,NaN,309.449703,NaN,17.480683,47.711980,NaN,NaN,NaN,NaN,NaN,3.649939,26.084551,5.560365,NaN,NaN,3.333047,3.959279,NaN,NaN,NaN,NaN,NaN,NaN,3.163079,1.166058,NaN,NaN,NaN,NaN,NaN,668.274699,NaN,98.935487,0.300641,NaN,32.562055,NaN,1.535195,NaN
1,2016-01-01T00:00:00,47,7,410.584193,0,1,1,100,1,1,1,6,0,1,1,0,0,2,0,0,0,85,1,0,0,0,0,748.584518,618.059831,42.9,-8.0,18.3,1.6,0.0,148.568863,NaN,NaN,NaN,NaN,NaN,NaN,282.533648,NaN,4.952174,89.995431,NaN,NaN,34.230919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.502762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.276033,NaN,NaN,NaN
2,2016-01-01T00:00:00,53,7,532.252963,1,4,3,80,1,1,5,6,0,0,1,0,1,1,0,0,0,70,1,0,0,0,0,749.407138,1343.725829,42.9,-8.0,18.3,1.6,0.0,281.710458,9.829813,7.184699,NaN,57.132283,NaN,41.131018,377.370916,24.187589,116.455238,56.500691,NaN,NaN,NaN,5.838335,NaN,4.818217,NaN,NaN,NaN,NaN,69.523585,NaN,NaN,1.580258,23.526942,27.425978,NaN,9.472365,70.962960,NaN,NaN,NaN,NaN,NaN,NaN,111.840309,NaN,NaN,NaN,NaN,47.234176,NaN,NaN,NaN
3,2016-01-01T00:00:00,67,7,385.268840,1,4,2,85,1,1,5,4,1,1,1,0,0,1,0,0,1,25,1,0,0,1,0,370.179157,295.014190,42.9,-8.0,18.3,1.6,0.0,17.143372,NaN,NaN,NaN,NaN,NaN,106.940645,NaN,NaN,5.528011,41.501197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.016888,52.845277,NaN,NaN,NaN,NaN,NaN,16.407566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.016997,NaN,NaN,3.614237,NaN,NaN,NaN,NaN,NaN
4,2016-01-01T00:00:00,68,7,1101.175074,0,1,1,90,1,1,5,5,1,2,2,0,1,1,0,1,1,43,1,0,0,1,0,2303.336977,2435.025827,42.9,-8.0,18.3,1.6,0.0,152.499806,3.707276,NaN,175.776547,140.411967,14.97116,NaN,254.280259,NaN,5.108473,120.966323,NaN,NaN,16.449692,NaN,NaN,27.656634,NaN,33.362191,NaN,NaN,126.670105,NaN,NaN,18.222686,55.938184,55.719367,NaN,214.492456,285.587951,NaN,NaN,80.205484,NaN,NaN,NaN,319.334544,159.860498,73.549838,NaN,NaN,33.530006,NaN,66.724382,NaN


# Diccionario de Variables del Datamart

Este documento describe las variables contenidas en el datamart de viviendas y hogares, utilizadas para el análisis de consumo ajustado por inflación y características del entorno.

---

## 🆔 Identificación del registro

- **`Timestamp`**: Año en el que se recogió la muestra, en formato ISO 8601.  
  Ejemplo: `2016-01-01T00:00:00`. Rango: 2016 a 2023.
- **`NUMERO`**: Identificador único de la vivienda en ese año.  
  Forma clave primaria junto a `Timestamp`.
- **`CCAA`**: Código oficial de la comunidad autónoma.  
  Rango de valores: 1 a 19.

---

## 📊 Ponderación muestral

- **`FACTOR`**: Factor poblacional de expansión.  
  Representa cuántos hogares reales representa ese hogar en la muestra.

---

## 🌍 Información geográfica

- **`CAPROV`**: Indica si la vivienda está en una capital de provincia.  
  `1` → Sí, `0` → No.
- **`TAMAMU`**: Tamaño del municipio.  
  `5` → ≥100.000 hab.  
  `4` → 50k–100k  
  `3` → 20k–50k  
  `2` → 10k–20k  
  `1` → <10k
- **`DENSIDAD`**: Densidad de población en la zona.  
  `3` → Densamente poblada  
  `2` → Intermedia  
  `1` → Diseminada
- **`SUPERF`**: Superficie útil de la vivienda.  
  `'35'` → ≤35 m²  
  `36–299` → Valor real  
  `'300'` → ≥300 m²  
  `-9` → No consta
- **`ZONARES`**: Tipo de zona de residencia.  
  `7` → Urbana de lujo  
  `6` → Urbana alta  
  `5` → Urbana media  
  `4` → Urbana inferior  
  `3` → Rural industrial  
  `2` → Rural pesquera  
  `1` → Rural agraria  
  `-9` → No consta

---

## 🏠 Características del hogar

- **`AGUACALI`**: Disposición de agua caliente.  
  `1` → Sí · `0` → No
- **`CALEF`**: Disposición de calefacción.  
  `1` → Sí · `0` → No
- **`REGTEN`**: Régimen de tenencia de la vivienda.  
  `6` → Propiedad sin hipoteca  
  `5` → Propiedad con hipoteca  
  `4` → Alquiler  
  `3` → Alquiler reducido  
  `2` → Cesión semigratuita  
  `1` → Cesión gratuita

---

## 👨‍👩‍👧 Composición del hogar

- **`NUMOCU`**: Número de miembros ocupados en el hogar (0–20).
- **`NUMACTI`**: Número de miembros activos en el hogar.
- **`NUMPERI`**: Número de perceptores de ingresos en el hogar (0–20).
- **`NUMESTU`**: Número de estudiantes en el hogar (0–20).
- **`NADUL_MAS`**: Número de adultos masculinos en el hogar.
- **`NADUL_FEM`**: Número de adultos femeninos en el hogar.
- **`NNINO_MAS`**: Número de niños masculinos en el hogar.
- **`NNINO_FEM`**: Número de niños femeninos en el hogar.

---

## 🧍 Datos del sustentador principal

- **`OCUSP`**: Ocupación del sustentador principal.  
  `1` → Ocupado · `0` → No ocupado
- **`EDADSP`**: Edad del sustentador principal en años.  
  `85` → 85 o más años  
  `16–84` → Edad real entre 16 y 84
- **`NACION_ESP`**: Nacionalidad del sustentador principal.  
  `1` → Española · `0` → Extranjera
- **`EDUC_SUPERIOR`**: Estudios superiores completados por el sustentador principal.  
  `1` → Sí · `0` → No
- **`CAPROP`**: Ingresos del sustentador principal por cuenta propia.  
  `1` → Sí · `0` → No
- **`CAJENA`**: Ingresos por cuenta ajena.  
  `1` → Sí · `0` → No
- **`DISPOSIOV`**: Disposición de otras viviendas en los últimos 12 meses.  
  `1` → Sí · `0` → No

---

## 💰 Indicadores económicos

- **`IMPEXAC`**: Importe exacto de los ingresos mensuales netos totales del hogar, ajustado a precios constantes de 2023.
- **`GASTOT_MENSUAL`**: Estimación de gasto total mensual del hogar, ajustado a precios constantes de 2023.

---

## 🌡️ Variables de características del entorno

- **`Tmax_max`**: Temperatura máxima media anual de la comunidad autónoma (ºC).
- **`Tmin_min`**: Temperatura mínima media anual de la comunidad autónoma (ºC).
- **`Tasa_Paro`**: Tasa de paro en la comunidad autónoma (%) correspondiente al año de la encuesta.
- **`Inflacion`**: Tasa de inflación interanual (%) en la comunidad autónoma.
- **`Tipo_Interes`**: Tipo de interés medio anual del Banco Central Europeo (%).

---

## 📤 Variables de output

Estas variables representan el **gasto mensual del hogar en cada una de las categorías de consumo**, ya **ajustado a precios constantes de 2023** según la tasa de inflación anual por comunidad autónoma.

El nombre de cada variable sigue el formato:  
**`NOMBRE_DE_LA_CATEGORÍA_CODIGO`**, donde:

- `NOMBRE_DE_LA_CATEGORÍA` es una versión en mayúsculas del nombre oficial de la categoría ECOICOP, con espacios reemplazados por guiones bajos `_`.
- `CODIGO` es el identificador numérico de la supercategoría.

### 🧾 Ejemplos de variables de output:

- `PRODUCTOS_ALIMENTICIOS_11`
- `BEBIDAS_NO_ALCOHÓLICAS_12`
- `BEBIDAS_ALCOHÓLICAS_21`
- `TABACO_22`
- `ARTÍCULOS_DE_VESTIR_31`
- `CALZADO_32`
- `ALQUILERES_REALES_DE_LA_VIVIENDA_41`
- `ALQUILERES_IMPUTADOS_DE_LA_VIVIENDA_42`
- `MANTENIMIENTO_DE_LA_VIVIENDA_43`
- `SUMINISTRO_DE_AGUA_44`
- `ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_45`
- `MUEBLES_REVESTIMIENTOS_51`
- `TEXTILES_PARA_EL_HOGAR_52`
- `GRANDES_ELECTRODOMÉSTICOS_53`
- `UTENSILIOS_DEL_HOGAR_54`
- `HERRAMIENTAS_CASA_JARDÍN_55`
- `BIENES_SERVICIOS_PARA_EL_HOGAR_56`
- `PRODUCTOS_FARMACÉUTICOS_61`
- `SERVICIOS_MÉDICOS_AMBULATORIOS_62`
- `SERVICIOS_HOSPITALARIOS_63`
- `COMPRA_DE_VEHÍCULOS_71`
- `USO_DE_VEHÍCULOS_PERSONALES_72`
- `SERVICIOS_DE_TRANSPORTE_73`
- `SERVICIOS_POSTALES_81`
- `EQUIPOS_TELÉFONO_FAX_82`
- `SERVICIOS_TELÉFONO_FAX_83`
- `AUDIOVISUALES_TECNOLOGÍA_91`
- `BIENES_DURADEROS_DE_OCIO_92`
- `OCIO_JARDINERÍA_MASCOTAS_93`
- `SERVICIOS_RECREATIVOS_CULTURALES_94`
- `PRENSA_PAPELERÍA_95`
- `PAQUETES_TURÍSTICOS_96`
- `EDUCACIÓN_INFANTIL_PRIMARIA_101`
- `EDUCACIÓN_SECUNDARIA_POSTSECUNDARIA_102`
- `EDUCACIÓN_SUPERIOR_103`
- `EDUCACIÓN_NO_FORMAL_104`
- `RESTAURACIÓN_111`
- `ALOJAMIENTO_112`
- `CUIDADOS_PERSONALES_121`
- `EFECTOS_PERSONALES_123`
- `PROTECCIÓN_SOCIAL_124`
- `SEGUROS_125`
- `SERVICIOS_FINANCIEROS_126`
- `OTROS_SERVICIOS_127`
- `REMESAS_128`

Cada una de estas columnas representa el **importe medio mensual**, en euros, que el hogar ha destinado a dicha categoría.

> 📎 **Nota**: Para más información técnica y definiciones oficiales sobre las categorías de gasto, puede consultar el **anexo PDF** proporcionado por el INE:  
anexo_codes.pdf

In [259]:
# Guardar el DataFrame final en un archivo TSV
def save_final_dataframe(df_final, output_path):
    """
    Guarda el DataFrame final en un archivo TSV.
    """
    df_final.to_csv(output_path, sep="\t", index=False)

In [262]:
output_path = "datamarts/final_dataframe.tsv"

save_final_dataframe(df_final, output_path)